# Netflix Shows
Introduction thing

## Data Description

Our dataset is a collection of information about Netflix shows as of 2019. Each obsevation contains information about a Netflix show's unique ID, its type (i.e. whether it is a TV Show or a movie), title, director, cast, country (where it was produced), release date, release year, TV rating, duration, genre, and description.

The dataset 'Netflix Movies and TV Shows' was collected from Flixable, a third-party Netflix search engine. Flixable was created in 2018 by Ville Salminen, and it came with additional advanced search functionality which was missing from the implemented search engine of Netflix. The data was extracted from the Flixable database through the use of API calls.

Currently, Netlflix does not have their API publicly available, and Flixable has not openly disclosed how the web site was able to acquire the data for its database. Despite the popularity of the aforementioned site, we cannot confirm whether the dataset that was extracted from Flixable is reliable. Moreover, considering that the latest update for the dataset was on November 2019, the conclusions made in this case study may not be representative of the current Netflix shows as of September, 2020.

## Exploratory Data Analysis

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rule_miner import RuleMiner
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import sklearn

Something load dataset

In [ ]:
netflix_df = pd.read_csv("./netflix_titles.csv")
netflix_df.head()

In [ ]:
netflix_df['director'].unique()

Remove duplicates dito ba?? ewan

#### Checking for `NAN`s

In [ ]:
netflix_df.isnull().any()

We see here that the `description` and `listed_in` variables do not have `Nan`/`null` values. Hence, no need to drop rows (for this question, at least) or do imputation. 

#### Checking for duplicates

The `description` variable is a good feature to check for duplicates as the synopsis are expectedly unique for each show.

In [ ]:
netflix_desc_duplicate = netflix_df['description'].value_counts().reset_index(name="description").query("description > 1")
print(netflix_desc_duplicate)
print("Repeated descriptions:", len(netflix_desc_duplicate))
print("Repeated shows:", netflix_desc_duplicate['description'].sum())

There are 7 repeated descriptions, and a total of 15 repeated shows based on our assumption that observations with the same description are the same show. To further confirm this, we can look at the other variables of the observations corresponding to these descriptions.

In [ ]:
netflix_df.loc[netflix_df["description"].isin(netflix_desc_duplicate["index"])]

Closely examining the observations reveals that the only differences in the observations are the `date_added` and the `title` variables. The date when the show is added doesn't give much insight as it does not give relevant context to the similarity of the show (i.e. different versions of a show can be added at different times, and vice versa). However, the title &mdash; at least in the results above &mdash; are descriptive in the differences.

The titles show that there are multiple versions due to langauge. The show *The Incredibles 2*, for instance, has both the original, and the Spanish version. Meanwhile, the movie *Oh! Baby* has two other versions: the Malalayam and Tamil. The only observation which didn't show any difference was for the movie *Sarkar* whose titles (there are 2 observations for this movie) do not indicate versions. This may either be a data collection error, or the title simply does not show which version it is.

Nonetheless, all the duplicates will be cleaned in the same way. The "original" version (i.e. the one without a parenthesis stating the version, or if not applicable, the first entry that appears in the dataset for simplicity) will be retained. 

This step is important because duplicates will affect the weight of the words during feature extraction. Thus, there would be bias for a particular show which would produce results that are not representative of the show's listed genre.

Because there are only few duplicates, we can manually remove the offending observations via `show_id`.

In [ ]:
show_id_remove_list = [81186758, 81186757, 81072516, 81046962, 81059388, 81151877, 81074135, 81091424]

netflix_df.loc[(netflix_df["description"].isin(netflix_desc_duplicate["index"])) & (~netflix_df["show_id"].isin(show_id_remove_list))]

The above shows the observations that will be retained from all the duplicates.

In [ ]:
netflix_df_clean = netflix_df[~netflix_df["show_id"].isin(show_id_remove_list)]
netflix_df_clean

## Exploratory questions:

#### 1.) What genres appear most frequently among Netflix shows?

For our first exploratory question, we decided to determine what genres are the most frequent among Netflix shows.

The genres of each Netflix show is shown in the `listed_in` column of the dataset in a single string format. Each string can contain more than one genre, and they are spearated by commas. To be able to count the frequency of each genre in the dataset, the `listed_in` column must be parsed into another table with each row containing all genres of a show, and each column containing only a single genre.

First we take the `listed_in` column of the dataset and isolate the genres per Netflix show by splitting the strings at commas with spaces after them (, ).

In [ ]:
listed_in_series = netflix_df_clean['listed_in']
genre_matrix = []

for string in listed_in_series:
    split_str = string.split(', ')
    genre_matrix.append(split_str)

genre_df = pd.DataFrame(genre_matrix)
genre_df

This results in a table with 6226 observations and 3 columns, which means that the maximum number of genres that netflix shows are listed in is only 3. Knowing this, we can create a series by concatenating each column into one single column. Then we can drop the values which are 'None', thus cleaning the resulting series

In [ ]:
genre_list = pd.concat([genre_df[0], genre_df[1], genre_df[2]])
genre_list.dropna(inplace = True)
genre_list

Now that the genres have been isolated, we can now use a bar plot to visualize the number of times a particular genre appeared on a Netflix show.

In [ ]:
genre_count = genre_list.value_counts()

genre_count.plot.bar()
plt.xlabel('Genre')
plt.ylabel('Count')
plt.title('Bar plot of genre count in Netflix shows')
genre_count

In the bar plot, we can see the ranking of the shows based on its frequency in Netflix shows. We see that 'International Movies' appear most frequently with almost 1922 appearances in the dataset.

#### 2.) What is the trend on the number of International Movies per year 

As we can see from the bar plot of the genre frequency, 'International Movies' was determined to be the most frequent genre in the dataset. Knowing this, the second exploratory question will focus on the trend of 'International Movies' according to its release year in the Netflix dataset. First we need to create a dataframe which only contain the shows which are listed as 'International Movies' in the dataset. We also no longer need to perform additional data cleaning, since the dataset has no null values for `listed_in` or `release_year`.

In [ ]:
netflix_df_international = netflix_df_clean
netflix_df_international.insert(12, "isInternationalMV", netflix_df_clean["listed_in"].str.find("International Movies") != -1, True)
netflix_df_international = netflix_df_international[netflix_df_international.isInternationalMV]
netflix_df_international

Now that we have a dataframe which only contains the shows which have been listed as 'International Movies', we can now use a histogram in order to visualize the trend of shows listed as 'International Movies' in the Netlflix dataset

In [ ]:
netflix_df_international['release_year'].hist(bins=60)
plt.xlabel('Release Year')
plt.ylabel('Count')
plt.title('Histogram of International Movies per year')

Based on the histogram above, we can see that within the Netflix catalog, there is a large number of shows listed as 'International Movies' which were released towards the latter half of the 2010's. 

#### 3.) What countries produce the most 'International Movies'?

Considering that our previous exploratory question answered that shows listed as 'International Movies' were the most prominent in the Netflix dataset, the third exploratory question would determine which countries these shows belong to. 

Since we already have a dataframe which contains all the shows listed as 'International Movies', we can use that dataframe for our analysis.

In [ ]:
netflix_df_international['country']

As we can see, the `country` column reveals that there are some instances in which a show was produced in more than one country and these countries are separated by commas. As such, we need to parse the `country` column into another table with each row containing all countries of a show, and each column containing only a single country

In [ ]:
netflix_df_international_country = netflix_df_international.dropna(subset=['country'])
international_series = netflix_df_international_country['country']
international_matrix = []

for string in international_series:
    split_str = string.split(', ')
    international_matrix.append(split_str)

international_df = pd.DataFrame(international_matrix)
international_df

This results in a table with 1854 observations and 11 columns, which means that the maximum number of countries that the shows in our dataframe were produced in is 11. As such, we can create a series by concatenating each column into one single column. Then we can drop the values which are 'None', thus cleaning the resulting series

In [ ]:
country_list = pd.concat([international_df[0], international_df[1], international_df[2], international_df[3], international_df[4], international_df[5], international_df[6], international_df[7], international_df[8], international_df[9], international_df[10], international_df[11]])
country_list.dropna(inplace = True)
country_list

Now that the countries have been separated, we can now use a bar plot to visualize the distribution of shows listed as 'International Movies' based on the country where it was produced

In [ ]:
country_count = country_list.value_counts()

country_count.plot.bar()
plt.xlabel('Country')
plt.ylabel('Count')
plt.title('Bar plot of distribution of International Movies based on country')
country_count

## Research Question

## Data Modelling

### What are the most common association rules among Netflix genres?

The `listed_in` column in the dataset refers to what genres the shows are listed in on the streaming platform. The values in this column are represented as strings where each string can contain more than one genre, and each genre is separated by commas. To parse these genres we need to create a matrix that is a list of genre lists, where each row represents the genres of one show. We can then use this matrix to create a dataframe that we can manipulate.

In [ ]:
listed_in_series = netflix_df_clean['listed_in']
genre_matrix = []

for string in listed_in_series:
    split_str = string.split(', ')
    genre_matrix.append(split_str)

genre_df = pd.DataFrame(genre_matrix)
genre_df

We also need a value dictionary to assign a number to each genre.

In [ ]:
values = genre_df.values.ravel()
values = [value for value in pd.unique(values) if not pd.isnull(value)]

value_dict = {}
for i, value in enumerate(values):
    value_dict[value] = i
value_dict

After generating the dictionary, we can then create a dataframe where each row is a netflix show and each column is a genre. If the show is listed in that genre then the value for that column is `1` and `0` if it is not

In [ ]:
genre_df = genre_df.stack().map(value_dict).unstack()

baskets = []
for i in range(genre_df.shape[0]):
    basket = np.sort([int(x) for x in genre_df.iloc[i].values.tolist() if str(x) != 'nan'])
    baskets.append(basket)

shows_df = pd.DataFrame([[0 for _ in range(len(value_dict))] for _ in range(len(baskets))], columns=values)

for i, basket in enumerate(baskets):
    shows_df.iloc[i, basket] = 1
shows_df

After preparing the dataframe, we can finally use a **Rule Miner** to figure out the common genre association rules of netflix shows. There can be many configurations of this Rule Miner that will produce different results. Because of this, multiple tests will be done on the dataset which will fulfill different purposes.

The first configuration will be setting the support threshold to the **lowest occurence** of a genre. This will be done to accomodate all genres in the dataset since all genres have appeared that many times. To do this we must first get all the total counts of each genre, then get the lowest value from the results. Since the total count of each genre has already been generated in the Exploratory Data Analysis, all we need to do is get the minimum value from that series.

In [ ]:
print("Last 10 results of the series\n")
print(genre_count.tail(10),"\n")
print("Minimum no. of occurrences:", genre_count.min())

The minimum value of the total genre counts is `10`, which will be set as the support threshold.

The confidence level is then lowered starting from `100%` until it produces at least 1 association rule

From this result, we initialize the first Rule Miner to have a **support threshold** of `10` and a **confidence** of `100%`.

In [ ]:
rule_miner_lowest_occur = RuleMiner(10, 1)
assoc_rules_lowest_occur = rule_miner_lowest_occur.get_association_rules(shows_df)
assoc_rules_lowest_occur

From the result of the get_association_rules function of the rule miner, we can see that the genre association rules are:
 - ['Crime TV Shows', 'Korean TV Shows'] → ['International TV Shows']
 - ['International TV Shows', 'Science & Nature TV'] → ['Docuseries']
 - ['Science & Nature TV', 'International TV Shows'] → ['Docuseries']
 - ['Romantic TV Shows', 'Korean TV Shows'] → ['International TV Shows']
 - ['TV Dramas', 'Korean TV Shows'] → ['International TV Shows']
 - ['British TV Shows', 'Science & Nature TV'] → ['Docuseries']

The second configuration will be setting the support threshold to the **average occurence** of all genres. This will be done to make sure that the genres that will be included in the rules must have occured a sufficient number of times. We can reuse the total counts of each genre and get the mean value from the series.

In [ ]:
print("Average no. of occurrences:", genre_count.mean())

The minimum value of the total genre counts is `324.93`, which we can round off and set as the support threshold.

The confidence level is then lowered starting from `100%` until it produces at least 1 association rule

From this result, we initialize this Rule Miner to have a **support threshold** of `325` and a **confidence** of `74%`.

In [ ]:
rule_miner_mean_occur = RuleMiner(325, 0.74)
assoc_rules_mean_occur = rule_miner_mean_occur.get_association_rules(shows_df)
assoc_rules_mean_occur

From the result of the get_association_rules function of the rule miner, we can see that the only genre association rule produced is:
 - ['Independent Movies'] → ['Dramas']

In [ ]:
# test code only

rule_miner_mean_occur = RuleMiner(325, 0.60)
assoc_rules_mean_occur = rule_miner_mean_occur.get_association_rules(shows_df)
assoc_rules_mean_occur

### Are Netflix synopses informative enough to classify whether shows belong to the International Movies genre?

The `description` variable in the dataset refers to Netflix's synopsis of each show. For this research question, the idea is to determine whether these synopses are informative enough to classify whether a show is classified under *International Movies*. This entails building a classifier which would be able to identify all *International Movie* shows in the dataset.

This is interesting because the problem leads to more questions about how informative Netflix synopses are. Are plots &mdash; represented by the synopses &mdash; dependent of genre? Are stories similar for the same genre? Are words able to represent genres? These will not be answered in this research problem, but are indications of the many stimulating possibilities that this research problem entails. It's also an indication of the capabilities of computers in handling natural language.

The *International Movies* genre is chosen because 1,922 shows are classified under this genre which makes it the most prominent. Choosing a prominent genre is important because there is a need to have a good amount of samples; this is especially true for classification which requires good representatives for each categories &mdash; in this case, the categories are "International Movies" and "Not International Movies". For this problem, only the `description` and `listed_in` variables from the Netflix dataset will be used. 

### Data Preparation

For this classification, two sets of data are needed: data which contains only shows with *International Movies* as its genre (regardless whether it's also classified as other genres), and data containing shows which are not classified under *International Movies*.

In the exploratory data analysis, there's already a `DataFrame` pertaining to the first. What we need is a `DataFrame` for shows that aren't classified under *International Movies*. To do this, the `isInternationalMV` variable from `netflix_df_clean`, can be used where False values indicates that show is not under said classification. After getting the False values, the next step is to concatenate both "Not International Movies" and "International Movies" data together to prepare it for feature extraction.

In [ ]:
netflix_df_non_international = netflix_df_clean[netflix_df_clean["isInternationalMV"] == False]
netflix_df_non_international

In [ ]:
netflix_df_classifier = pd.concat([netflix_df_non_international, netflix_df_international])
netflix_df_classifier

In [ ]:
print("International Movies:", len(netflix_df_international))
print("Not International Movies:", len(netflix_df_non_international))

- **Total International Movies shows:** 1922 shows

- **Total Not International Movies shows:** 4304 shows

### Feature Extraction
The `description` variable is in the form of sentences (naturally, because synopses are written with words). The task is to evaluate the value of words in these synposes.  However, computers do not really understand sentences. Hence, a mathematical model is important to transform the words into a numerical representation. 

#### TF-IDF
The term frequency-inverse document frequency (TF-IDF) is a statistical method which places value in words through an inverse proportion of the word's frequency in a document, in this case the `description`, to the amount of documents the word appears in. The importance of a word is indicated by a high TF-IDF value. The TF-IDF value is given by the formula: 

$$W_{i,j} = tf_{i,j} \times log{N \over df_{i}}$$

Where:

$tf_{i,j}$ = number of occurences of $i$ in $j$,

$df_{i}$ = number of documents containing $i$, and

$N$ = total number of documents

Scikit-learn's [TF-IDF Vectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) was utilized to transform the `description` column into a matrix of TF-IDF features. First we fit the documents (i.e. the synposes) by using the vectorizer's `fit` function for the vectorizer to learn the vocabulary. Once vectorizer knows the vocabulary and document frequencies, the `transform` function is used to transform the documents into a document-term matrix.  

In [ ]:
vectorizer=TfidfVectorizer()
vectorizer.fit(netflix_df_classifier["description"])
features=vectorizer.transform(netflix_df_classifier["description"])
features

We see that the TF-IDF Vectorizer converted the `description` column into a sparse matrix of TF-IDF values where each row contains 16411 columns. This also means that there are 16411 words that were extracted as features from the dataset. To confirm this, we can get the word form of the features from the vectorizer by using its `get_feature_names` function.

In [ ]:
features_words = vectorizer.get_feature_names()
print(features_words)

In [ ]:
len(features_words)

The above is a list of all the words, sorted alphabetically, that the vectorizer learned and, consequently, used as features to build the sparse matrix. Results show that there are indeed 16411 features for each row (and thus, for each show).

### Preparing labels
In order for the classifier to classify, it should be trained knowing the labels &mdash; in this case *International Movie* or *Not International Movie* &mdash; of the show it's trying to classify. The next step is to prepare these labels by utilizing the `isInternationalMV` variable of the dataset to form an array of labels. *International Movie* shows are represented as 1s, while *Not International Movie* shows are represented as 0s.

In [ ]:
labels=[]
for show in netflix_df_classifier["isInternationalMV"]:
    if show == True:
        appenddata=1
    if show == False:
        appenddata=0
    labels.append(appenddata)
    
labels

Because *Not International Movie* shows and *International Movie* shows were simply concatenated, it's expected that the dataset is split homogenously where the first parts are all *Not International Movie* shows and the latter are *International Movie* shows. This is not a problem, however, since a stratified cross-validation approach will be utilized which splits the data such that there will be close-as-possible distribution of classes.

### Building a classifier and measuring performance
This [chart](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) from scikit-learn was utilized to choose the right estimator for this problem. The circumstances of this research lead us to selecting [`LinearSVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) from scikit-learn's support vector machine module.

Feeding the data to the machine algorithm can be manually done by using the functions of the `LinearSVC` class. However, as mentioned earlier, cross-validation will be used in evaluating the performance of the dataset. For this purpose,  [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) will allow us to both build the classifier, and evaluate its performance at the same time. For binary classification problems such as what we have, `cross_validate` automatically uses the [`StratifiedKFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold) as its cross-validation splitter, hence, we minimize having bias in each fold.

The first step is to initialize the estimator (Linear SVC), and name the metrics/scoring that will be used by the cross-validation in evaluating performance. Default parameters were used for the estimator.

In [ ]:
linearSVC=svm.LinearSVC()
scoring = ['accuracy', 'recall', 'precision']

Then, we simply use the estimator, the features, the labels, and the scorers as parameters for `cross_validate`.

In [ ]:
scores = cross_validate(linearSVC, features, labels, scoring=scoring)
print('Mean Accuracy:', scores['test_accuracy'].mean())
print('Accuracy Two Standard Deviations:', scores['test_accuracy'].std() * 2)
print('Mean Precision', scores['test_precision'].mean())
print('Mean Recall', scores['test_recall'].mean())

- **95% Confidence Interval of Genre Classifier Accuracy:** 72.23% $\pm$ 2.98

- **Average Genre Classifier Precision:** 57.83%

- **Average Genre Classifier Recall:** 39.80%

To get more information, we can extract the top features by using [`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selectkbest#sklearn.feature_selection.SelectKBest) which returns the top *k* features based on a scoring function. In this case, we will use [`Chi-squared statistics`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2)because it is expected that the features &mdash; in the form of TF-IDF values &mdash; will never be negative. Furthermore, because chi-squared statistics measures dependence between variables, it can be used to determine which features are independent of classes, hence, not useful in classification. 

### Top features for classification

In [ ]:
selector = SelectKBest(chi2, k=20)
selector.fit(features, labels)
# Get idxs of columns to keep
top_features_idxs = selector.get_support(indices=True)

scores, pval=(chi2(features, labels))

chi2_data={'Feature Names': np.array(features_words)[top_features_idxs], 'Chi2': scores[top_features_idxs], 'P-value': pval[top_features_idxs]}
features_chi2 = pd.DataFrame(chi2_data).sort_values(by=['Chi2'], ascending=False, ignore_index=True)
features_chi2

The `DataFrame` shows the top 20 most class-dependent features identified through chi-squared statistics. When the classifier sees these words in the synopses, there is a high probability that it would be able to classify whether the show is an *International Movie* or not correctly.

Note, however, that the results are based on the evaluation of the data as a whole (i.e. did not follow the cross-validation scheme), and without consideration of the machine learning algorithm. The implication is that it may not be representative of the the accuracy of the classifiers that were measured during cross-validation. Nevertheless, this method still shows a general idea of what the top features would be for a classifier that is built with synopses and genre data from the Netflix dataset. 

### Insights and Conclusion

#### Accuracy
The 95% confidence interval for a Netflix genre classifier's accuracy is 72.23% $\pm$ 2.98. This means that the classifier, more than half of the time, was able to classify whether a show is an *International Movie* or *Not International Movie*. Only considering accuracy, this can be interpreted as successful.

However, even though a stratified cross-validation was used, there is still some class imbalance because the dataset contains 1,922 *International Movie* shows, and 4,304 *Not International Movie* shows. The latter class is more than twice the other. This renders the accuracy measure an inferior evaluator because it can yield a high accuracy even if it fails to classify a lot of International Movie shows.

For instance, if the classifier were to correctly guess all *Not International Movie* shows:

In [ ]:
intl_total = 1922
non_intl_total = 4304
    
non_intl_total / (intl_total + non_intl_total)

The accuracy would be 69.13% which can still be interpreted as the classifier being able to correctly classify more than half the time. Given so, we can look at [precision and recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall). The former is concerned with the correct *International Movie* show predictions, while the latter is concerned with the number of actual *International Movie* shows that were identified by the classifier.

#### Precision and Recall
The average precision for the classifiers is 57.83%. This means that more than half the time, when the classifiers predict a show to be under *International Movies*, it is true. Meanwhile, the average recall for the classifiers is 39.80%. This means that the classifiers were only able to identify less than half of all the *International Movie* shows. Given a standard of 50%, the classifiers were not successful at all.

Given that the goal was to correctly identify all *International Movies* shows from the data, precision is much less important than recall. As such, looking at recall, we can say that although accuracy is high, the classifiers are not able to satisfactorily identify the *International Movie* shows.

However, this means that accuracy is truly affected by the class imbalance. The implication is that the classifier were probably able to classify *Not International Movie* shows, at least better than its counterpart. To test this we can flip the labels to say that the true positives are the *Not International Movie* shows.

In [ ]:
invert_labels=[]
for show in netflix_df_classifier["isInternationalMV"]:
    if show == True:
        appenddata=0
    if show == False:
        appenddata=1
    invert_labels.append(appenddata)
    
invert_labels

In [ ]:
invert_scores = cross_validate(linearSVC, features, invert_labels, scoring=scoring)
print('Mean Precision', invert_scores['test_precision'].mean())
print('Mean Recall', invert_scores['test_recall'].mean())

- **Average Precision:** 76.34%

- **Average Recall:** 86.71%

The results show an astounding increase in performance. Recall increased more than twice which indicates that the classifiers were able to predict most of the *Not International Movie* shows in the dataset. 

Thus, it can still be said that synopses are still informative in genre classification, just the other way around. In future studies, a better way to test this is to have a dataset that have equal, or close distribution of classes.

#### Top Features
The top class-dependent features also gives insight about the performance of the classifier.

In [ ]:
features_chi2

Looking at the table, there words that can be identified to match with the idea of an international movie:
- Bollywood
- India
- Indian
- Mumbai
- Caste

Under a 0.05 significance level, these are all hugely relevant to the classification process. It's also worth noting that these words are all related to India. This result is consistent with the findings in the exploratory data analysis stating that most of the shows that fall under the *International Movies* genre &mdash; 714 shows to be exact &mdash; is produced in India.

In [ ]:
country_count

Thus, it is expected that the chi-squared statistics, given a set of *International Movies* shows, most of which are produced in India, would find Indian-related words to be relevant features for classification. 

#### Conclusion

Therefore, based on analysis, it can be said that Netflix synopses are not informative enough in classifying if shows belong to the International Movies genre. However, if the goal was identifying the opposite, Netflix synopses prove to be informative. In this latter context, performance metrics reveal that the classifier was able to perform well given only show synopses and their corresponding genre. 

Meanwhile, a brief top-features analysis show that synopses contain information, specifically words, that are relevant to its genre, and consequently, the classification. 

In the future, it is recommended to conduct parameter tuning for the estimator to see whether there are better parameters that would yield better classifier performance; a better performance may conclude that the problem was not due to the dataset, but with the classifier's setup. Another recommendation is to consider other metrics such as the confusion matrix to further analyze results and why the classifier behaves the way it does. It is also recommended that further data preprocessing techniques be done such as lemmatization, and stop words removal. Finally, in line with the chi-squared statistics of finding the top features, further analysis can be done to find out which words can describe *International Movies* in Netflix. 